In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge

In [ ]:
def interactive_plot(df, title):
  fig = px.line(title=title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

In [ ]:
def stock_predictor(proportion, ticker = "ticker", start_date= "start_date", end_date="end_date"):
  # Step 1: grab data
  df = yf.download(tickers=ticker, start=start_date, end=end_date)
  
  # Step 2: create our target, e.g want to predict next days stock price
  df['Target'] = df['Adj Close'].shift(-1)

  # Step 3: prepare data
  data = pd.DataFrame({'Adj Close': df['Adj Close'],'Volume': df['Volume'], 'Target': df['Target'] })[:-1]

  # Scaling data, easier for model to learn
  scale_range = MinMaxScaler(feature_range = (0,1))
  data_scaled = scale_range.fit_transform(data)
  
  # Setting up what proportion of data will be used for training/testing
  X, y = data_scaled[:, :2], data_scaled[:, 2:]
  split = int(proportion * len(X))
  X_train, y_train = X[:split], y[:split]
  X_test, y_test = X[split:], y[split:]

  # Step 4: train model
  regression_model = Ridge()
  regression_model.fit(X_train, y_train)

  # Step 5: evaluation and result
  linear_regression_accuracy = regression_model.score(X_test, y_test)

  scaled_adjclose = data_scaled[:,0].tolist()

  predicted_prices = regression_model.predict(X)[:,0].tolist()

  data_prediction = pd.DataFrame({'Date': df.index[:-1],'Scaled Adj Close': scaled_adjclose, 'Prediction': predicted_prices})

  return print('Ridge Regression Score:', linear_regression_accuracy), interactive_plot(data_prediction, 'Original vs Prediction')